In [2]:
import numpy as np
import sys
import os

# --- パスを通してモジュールをインポート可能にする ---
# GraphicalPR のルートを sys.path に追加
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# --- モジュールのインポート ---
from core.uncertain_array import UncertainArray

# --- テスト：UncertainArray のランダム生成 ---
shape = (4, 4)
ua = UncertainArray.random(shape)

# --- 確認出力 ---
print("Type:", type(ua))
print("Shape:", ua.shape)
print("Data dtype:", ua.data.dtype)
print("Precision shape:", ua.precision.shape)
print("Precision values (should all be 1.0):")
print(ua.precision)

# --- 検証 ---
assert ua.shape == shape
assert ua.data.shape == shape
assert ua.precision.shape == shape
assert np.allclose(ua.precision, 1.0)


Type: <class 'core.uncertain_array.UncertainArray'>
Shape: (4, 4)
Data dtype: complex128
Precision shape: (4, 4)
Precision values (should all be 1.0):
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


## multiplication

In [3]:
import sys, os
import numpy as np

# プロジェクトルートを sys.path に追加して core モジュールを import 可能にする
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from core import UncertainArray

# --- テスト対象の UncertainArray オブジェクトを生成 ---
data1 = np.array([1+1j, 2+2j])
data2 = np.array([3+3j, 4+4j])
precision1 = 1.0
precision2 = 3.0

ua1 = UncertainArray(data1, precision=precision1)
ua2 = UncertainArray(data2, precision=precision2)

# --- 積を計算 ---
ua3 = ua1 * ua2

# --- 期待値を計算 ---
expected_precision = precision1 + precision2
expected_data = (precision1 * data1 + precision2 * data2) / expected_precision

# --- 結果を出力 ---
print("ua3.data =", ua3.data)
print("expected =", expected_data)
print("ua3.precision =", ua3.precision)
print("expected precision =", expected_precision)

# --- アサーションで検証 ---
assert np.allclose(ua3.data, expected_data), "Data mismatch"
assert np.allclose(ua3.precision, expected_precision), "Precision mismatch"

print("✅ Test passed.")

ua3.data = [2.5+2.5j 3.5+3.5j]
expected = [2.5+2.5j 3.5+3.5j]
ua3.precision = [4. 4.]
expected precision = 4.0
✅ Test passed.


In [4]:
ua1 = UncertainArray([4+4j, 6+6j], precision=5.0)
ua2 = UncertainArray([2+2j, 3+3j], precision=3.0)
ua3 = ua1 / ua2

expected_precision = np.clip(5.0 - 3.0, 1.0, None)
expected_data = (5.0 * ua1.data - 3.0 * ua2.data) / expected_precision

assert np.allclose(ua3.data, expected_data)
assert np.allclose(ua3.precision, expected_precision)

In [5]:
import numpy as np
from core.uncertain_array import UncertainArray

# --- UA ランダム作成（スカラー精度） ---
ua1 = UncertainArray.random(shape=(2, 2), precision=2.0)
ua2 = UncertainArray.random(shape=(2, 2), precision=3.0)

print("ua1.data:\n", ua1.data)
print("ua2.data:\n", ua2.data)

# --- 掛け算テスト（スカラー精度） ---
ua_mul = ua1 * ua2
print("ua1 * ua2:\n", ua_mul.data)
print("precision (should be 5.0):", ua_mul.to_scalar_precision())

# --- 割り算テスト（スカラー精度） ---
ua_div = ua1 / ua2
print("ua1 / ua2:\n", ua_div.data)
print("precision:", ua_div.to_scalar_precision())

# --- 精度を配列に変更して掛け算 ---
ua1.set_precision([[2.0, 4.0], [6.0, 8.0]])
ua2.set_precision([[1.0, 2.0], [3.0, 4.0]])

ua_mul2 = ua1 * ua2
print("ua1 * ua2 with array precision:\n", ua_mul2.data)
print("precision (array):\n", ua_mul2.precision)

# --- combine テスト ---
ua3 = UncertainArray.random((2, 2), precision=5.0)
combined = UncertainArray.combine([ua1, ua2, ua3])
print("Combined data:\n", combined.data)
print("Combined precision:\n", combined.precision)

# --- scalar precision に丸める ---
scalar_prec = ua1.to_scalar_precision()
print("ua1.to_scalar_precision():", scalar_prec)


ua1.data:
 [[-0.67785956-1.47209277j  1.73480338-1.76989144j]
 [ 1.49005241-0.55972542j -1.55636151-1.01280078j]]
ua2.data:
 [[-0.41436363+0.01008708j  0.17970816+0.35117515j]
 [ 1.19709114+0.14276567j -0.31259728-0.06748483j]]
ua1 * ua2:
 [[-0.519762  -0.58278486j  0.80174625-0.49725149j]
 [ 1.31427565-0.13823077j -0.81010297-0.44561121j]]
precision (should be 5.0): 5.0
ua1 / ua2:
 [[-0.11262823-2.97444679j  2.93048227-4.59330833j]
 [-0.61116859-1.54774785j -2.17493116-1.82314706j]]
precision: 1.0
ua1 * ua2 with array precision:
 [[-0.59002758-0.97803282j  1.21643831-1.06286924j]
 [ 1.39239866-0.32556173j -1.14177343-0.69769546j]]
precision (array):
 [[ 3.  6.]
 [ 9. 12.]]
Combined data:
 [[-1.14883576-0.8740153j   1.01527533-1.28503158j]
 [ 1.16428626-0.43981924j -0.26954559-0.23316154j]]
Combined precision:
 [[ 8. 11.]
 [14. 17.]]
ua1.to_scalar_precision(): 3.8400000000000007
